# vLLM Handoff

## Imports

In [1]:
import base64
import wallaroo
import pyarrow as pa
import pandas as pd
from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.framework import Framework
from wallaroo.engine_config import Acceleration
from wallaroo.continuous_batching_config import ContinuousBatchingConfig
from wallaroo.object import EntityNotFoundError

In [2]:
wl = wallaroo.Client()

## Download & zip model

```bash
git lfs install
```

```bash
git clone https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct
```

```bash
cd ./Llama-3.1-8B-Instruct && rm -rf .git
```

```bash
cd .. & zip -r llama-31-8b-instruct.zip ./Llama-3.1-8B-Instruct
```

## Define Schemas & Upload model

In [3]:
input_schema = pa.schema([
    pa.field('prompt', pa.string()),
    pa.field('max_tokens', pa.int64()),
    # pa.field('top_p', pa.int64()),
    # pa.field('ignore_eos', pa.bool()),
    # pa.field('temperature', pa.int64()),
])
output_schema = pa.schema([
    pa.field('generated_text', pa.string()),
    pa.field('num_output_tokens', pa.int64())
])

### Upload model via API

In [ ]:
base64.b64encode(
    bytes(input_schema.serialize())
).decode("utf8")

In [ ]:
base64.b64encode(
    bytes(output_schema.serialize())
).decode("utf8")

Run the following command in order to upload the model via **API**:

```bash
curl --progress-bar -X POST   -H "Content-Type: multipart/form-data"   -H "Authorization: Bearer <your-auth-token-here>"   -F 'metadata={"name": "vllm-llama31-8b-async-fc-v3", "visibility": "private", "workspace_id": <your-workspace-id-here>, "conversion": {"framework": "vllm", "python_version": "3.8", "requirements": [], "framework_config": {"config": {"gpu_memory_utilization": 0.9, "max_model_len": 128}, "framework": "vllm"}}, "input_schema": "/////7AAAAAQAAAAAAAKAAwABgAFAAgACgAAAAABBAAMAAAACAAIAAAABAAIAAAABAAAAAIAAABUAAAABAAAAMT///8AAAECEAAAACQAAAAEAAAAAAAAAAoAAABtYXhfdG9rZW5zAAAIAAwACAAHAAgAAAAAAAABQAAAABAAFAAIAAYABwAMAAAAEAAQAAAAAAABBRAAAAAcAAAABAAAAAAAAAAGAAAAcHJvbXB0AAAEAAQABAAAAA==", "output_schema": "/////8AAAAAQAAAAAAAKAAwABgAFAAgACgAAAAABBAAMAAAACAAIAAAABAAIAAAABAAAAAIAAABcAAAABAAAALz///8AAAECEAAAACwAAAAEAAAAAAAAABEAAABudW1fb3V0cHV0X3Rva2VucwAAAAgADAAIAAcACAAAAAAAAAFAAAAAEAAUAAgABgAHAAwAAAAQABAAAAAAAAEFEAAAACQAAAAEAAAAAAAAAA4AAABnZW5lcmF0ZWRfdGV4dAAABAAEAAQAAAA="};type=application/json'   -F "file=@llama-31-8b-instruct.zip;type=application/octet-stream"   https://benchmarkscluster.wallaroocommunity.ninja/v1/api/models/upload_and_convert | cat
```

In [ ]:
# Retrieve the model
model = wl.get_model("vllm-llama31-8b-async-fc-v3")
model

### Upload model via SDK

In [4]:
from wallaroo.framework import VLLMConfig

In [17]:
??VLLMConfig

Init signature:
VLLMConfig(
    *,
    device_group: Union[List[int], NoneType, wallaroo.wallaroo_ml_ops_api_client.types.Unset] = <wallaroo.wallaroo_ml_ops_api_client.types.Unset object at 0x7b569d49cca0>,
    gpu_memory_utilization: Union[wallaroo.wallaroo_ml_ops_api_client.types.Unset, float] = 0.9,
    kv_cache_dtype: Union[wallaroo.wallaroo_ml_ops_api_client.types.Unset, wallaroo.wallaroo_ml_ops_api_client.models.kv_cache_dtype.KvCacheDtype] = <KvCacheDtype.AUTO: 'auto'>,
    max_model_len: Union[NoneType, wallaroo.wallaroo_ml_ops_api_client.types.Unset, int] = <wallaroo.wallaroo_ml_ops_api_client.types.Unset object at 0x7b569d49cd60>,
    max_num_seqs: Union[wallaroo.wallaroo_ml_ops_api_client.types.Unset, int] = 256,
    max_seq_len_to_capture: Union[wallaroo.wallaroo_ml_ops_api_client.types.Unset, int] = 8192,
    quantization: Union[wallaroo.wallaroo_ml_ops_api_client.types.Unset, wallaroo.wallaroo_ml_ops_api_client.models.quantization.Quantization] = <Quantization.NONE: 'none

In [6]:
model = wl.upload_model(
    "vllm-llama31-8b-async-demo", 
    "./vLLM_llama-31-8b.zip",
    framework=Framework.VLLM,
    framework_config=VLLMConfig(
        gpu_memory_utilization=0.9, 
        max_model_len=128
    ),
    input_schema=input_schema, 
    output_schema=output_schema,
    accel=Acceleration.CUDA
)
model

Please log into the following URL in a web browser:

	https://autoscale-uat-gcp.wallaroo.dev/auth/realms/master/device?user_code=VPKN-MCIQ

Login successful!
Waiting for model loading - this will take up to 10min.
.odel is pending loading to a container runtime.
.............................................successful

Ready


Name,vllm-llama31-8b-async-demo
Version,422d3ad9-1bc7-40c1-99af-0ba109964bfd
File Name,vLLM_llama-31-8b.zip
SHA,62c338e77c031d7c071fe25e1d202fcd1ded052377a007ebd18cb63eadddf838
Status,ready
Image Path,proxy.replicated.com/proxy/wallaroo/ghcr.io/wallaroolabs/mac-deploy:v2025.1.0-main-6132
Architecture,x86
Acceleration,cuda
Updated At,2025-08-May 19:24:36
Workspace id,60
Workspace name,younes.amar@wallaroo.ai - Default Workspace


In [7]:
model

Name,vllm-llama31-8b-async-demo
Version,422d3ad9-1bc7-40c1-99af-0ba109964bfd
File Name,vLLM_llama-31-8b.zip
SHA,62c338e77c031d7c071fe25e1d202fcd1ded052377a007ebd18cb63eadddf838
Status,ready
Image Path,proxy.replicated.com/proxy/wallaroo/ghcr.io/wallaroolabs/mac-deploy:v2025.1.0-main-6132
Architecture,x86
Acceleration,cuda
Updated At,2025-08-May 19:24:36
Workspace id,60
Workspace name,younes.amar@wallaroo.ai - Default Workspace


In [8]:
# Define continous batching for Async vLLM (you can choose the number of connections you want)
cbc = ContinuousBatchingConfig(max_concurrent_batch_size = 100)

In [9]:
batch = model.configure(
    input_schema = input_schema,
    output_schema = output_schema,
    continuous_batching_config = cbc
)

In [10]:
batch

Name,vllm-llama31-8b-async-demo
Version,422d3ad9-1bc7-40c1-99af-0ba109964bfd
File Name,vLLM_llama-31-8b.zip
SHA,62c338e77c031d7c071fe25e1d202fcd1ded052377a007ebd18cb63eadddf838
Status,ready
Image Path,proxy.replicated.com/proxy/wallaroo/ghcr.io/wallaroolabs/mac-deploy:v2025.1.0-main-6132
Architecture,x86
Acceleration,cuda
Updated At,2025-08-May 19:24:36
Workspace id,60
Workspace name,younes.amar@wallaroo.ai - Default Workspace


## Deployment

In [11]:
deployment_config = DeploymentConfigBuilder() \
    .cpus(1.).memory('1Gi') \
    .sidekick_cpus(batch, 1.) \
    .sidekick_memory(batch, '10Gi') \
    .sidekick_gpus(batch, 1) \
    .deployment_label("wallaroo.ai/accelerator:a100") \
    .build()

In [ ]:
pipeline = wl.build_pipeline("llama-31-8b-vllm-demo")
pipeline.clear()
pipeline.undeploy()

pipeline.add_model_step(batch)
pipeline.deploy(deployment_config=deployment_config)

In [13]:
pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.4.8.2',
   'name': 'engine-8558f6576d-8h7pc',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'llama-31-8b-vllm-demo',
      'status': 'Running',
      'version': '62806288-5f42-44b8-9345-bb4dfb613801'}]},
   'model_statuses': {'models': [{'model_version_id': 443,
      'name': 'vllm-llama31-8b-async-demo',
      'sha': '62c338e77c031d7c071fe25e1d202fcd1ded052377a007ebd18cb63eadddf838',
      'status': 'Running',
      'version': '422d3ad9-1bc7-40c1-99af-0ba109964bfd'}]}}],
 'engine_lbs': [{'ip': '10.4.1.17',
   'name': 'engine-lb-5cf49f9d5f-sqr4f',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': [{'ip': '10.4.8.7',
   'name': 'engine-sidekick-vllm-llama31-8b-async-demo-443-75d58845c-svvll',
   'status': 'Running',
   'reason': None,
   'details': [],
   'statuses': '\n'}]}

## Inference

In [14]:
data = pd.DataFrame({"prompt": ["What is Wallaroo.AI?"], "max_tokens": [200]})

In [15]:
pipeline.infer(data)

,time,in.max_tokens,in.prompt,out.generated_text,out.num_output_tokens,anomaly.count
0,2025-05-08 19:42:06.259,200,What is Wallaroo.AI?,Cloud and AutoML with Python\nWallaroo.AI is a...,122,0


In [16]:
pipeline.publish(deployment_config=deployment_config)

Waiting for pipeline publish... It may take up to 600 sec.
............................................... Published.


ID,36
Pipeline Name,llama-31-8b-vllm-demo
Pipeline Version,a5b7a202-9923-4d8d-ba4c-31e22a83cddc
Status,Published
Workspace Id,60
Workspace Name,younes.amar@wallaroo.ai - Default Workspace
Edges,
Engine URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini-cuda:v2025.1.0-main-6132
Pipeline URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/llama-31-8b-vllm-demo:a5b7a202-9923-4d8d-ba4c-31e22a83cddc
Helm Chart URL,oci://us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts/llama-31-8b-vllm-demo
Helm Chart Reference,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts@sha256:af38b73f10fbf6d9da318568d86383b762dee766547a35c30dccf5f7907695e1


## Undeploy

In [91]:
pipeline.undeploy()

Please log into the following URL in a web browser:

	https://autoscale-uat-gcp.wallaroo.dev/auth/realms/master/device?user_code=XXND-IZZK

Login successful!
 ok


name,llama-31-8b-vllm-ynsv5
created,2025-05-06 12:31:40.360907+00:00
last_updated,2025-05-06 19:51:47.490400+00:00
deployed,False
workspace_id,60
workspace_name,younes.amar@wallaroo.ai - Default Workspace
arch,x86
accel,cuda
tags,
versions,"b82ed30f-e937-4b49-94d5-63e6e798cc4b, b0a4ab4d-28ee-4470-9391-888a486375d2, 47760536-b263-428d-a9eb-f763c84f8920, 632917ff-0ffd-49be-abca-5a69a6432f93, 18cc0cad-cf6c-4abf-9083-ee90c2e704e2"
steps,vllm-llama31-8b-async-ynsv5
